In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 8.4 MB/s eta 0:00:00


In [ ]:
groq_api_key = "your_groq_api_key"

## **Generator**

In [ ]:
from groq import Groq
client = Groq(api_key = groq_api_key)

def call_llm_with_full_text(itext):
  text_input = '\n'.join(itext)
  prompt = f"Please elaborate on the folleoing content: \n{text_input}"

  try:
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",

        messages = [
            {"role": "system", "content": "You are an expert Natural Language Processing exercise expert."},
            {"role": "assistant", "content": "1.You can explain read the input and answer in detail"},
            {"role": "user", "content": prompt}
        ],
        temperature = 0.1
    )
    return response.choices[0].message.content.strip()

  except Exception as e:
    return str(e)

In [ ]:
import textwrap

# This function is used to wrap 80 characters per line

def print_formatted_response(response):
  wrapper = textwrap.TextWrapper(width=80)
  text_wrapped = wrapper.fill(text=response)

  print("Response -------------------")
  print(response)
  print("Formatted response -------------------")
  print(text_wrapped)

## **Data**

In [ ]:
db_records = [
    "Retrieval Augmented Generation (RAG) represents a sophisticated hybrid approach in the field of artificial intelligence, particularly within the realm of natural language processing (NLP).",
    "It innovatively combines the capabilities of neural network-based language models with retrieval systems to enhance the generation of text, making it more accurate, informative, and contextually relevant.",
    "This methodology leverages the strengths of both generative and retrieval architectures to tackle complex tasks that require not only linguistic fluency but also factual correctness and depth of knowledge.",
    "At the core of Retrieval Augmented Generation (RAG) is a generative model, typically a transformer-based neural network, similar to those used in models like GPT (Generative Pre-trained Transformer) or BERT (Bidirectional Encoder Representations from Transformers).",
    "This component is responsible for producing coherent and contextually appropriate language outputs based on a mixture of input prompts and additional information fetched by the retrieval component.",
    "Complementing the language model is the retrieval system, which is usually built on a database of documents or a corpus of texts.",
    "This system uses techniques from information retrieval to find and fetch documents that are relevant to the input query or prompt.",
    "The mechanism of relevance determination can range from simple keyword matching to more complex semantic search algorithms which interpret the meaning behind the query to find the best matches.",
    "This component merges the outputs from the language model and the retrieval system.",
    "It effectively synthesizes the raw data fetched by the retrieval system into the generative process of the language model.",
    "The integrator ensures that the information from the retrieval system is seamlessly incorporated into the final text output, enhancing the model's ability to generate responses that are not only fluent and grammatically correct but also rich in factual details and context-specific nuances.",
    "When a query or prompt is received, the system first processes it to understand the requirement or the context.",
    "Based on the processed query, the retrieval system searches through its database to find relevant documents or information snippets.",
    "This retrieval is guided by the similarity of content in the documents to the query, which can be determined through various techniques like vector embeddings or semantic similarity measures.",
    "The retrieved documents are then fed into the language model.",
    "In some implementations, this integration happens at the token level, where the model can access and incorporate specific pieces of information from the retrieved texts dynamically as it generates each part of the response.",
    "The language model, now augmented with direct access to retrieved information, generates a response.",
    "This response is not only influenced by the training of the model but also by the specific facts and details contained in the retrieved documents, making it more tailored and accurate.",
    "By directly incorporating information from external sources, Retrieval Augmented Generation (RAG) models can produce responses that are more factual and relevant to the given query.",
    "This is particularly useful in domains like medical advice, technical support, and other areas where precision and up-to-date knowledge are crucial.",
    "Retrieval Augmented Generation (RAG) systems can dynamically adapt to new information since they retrieve data in real-time from their databases.",
    "This allows them to remain current with the latest knowledge and trends without needing frequent retraining.",
    "With access to a wide range of documents, Retrieval Augmented Generation (RAG) systems can provide detailed and nuanced answers that a standalone language model might not be capable of generating based solely on its pre-trained knowledge.",
    "While Retrieval Augmented Generation (RAG) offers substantial benefits, it also comes with its challenges.",
    "These include the complexity of integrating retrieval and generation systems, the computational overhead associated with real-time data retrieval, and the need for maintaining a large, up-to-date, and high-quality database of retrievable texts.",
    "Furthermore, ensuring the relevance and accuracy of the retrieved information remains a significant challenge, as does managing the potential for introducing biases or errors from the external sources.",
    "In summary, Retrieval Augmented Generation represents a significant advancement in the field of artificial intelligence, merging the best of retrieval-based and generative technologies to create systems that not only understand and generate natural language but also deeply comprehend and utilize the vast amounts of information available in textual form.",
    "A RAG vector store is a database or dataset that contains vectorized data points."
]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer(
        stop_words='english',
        use_idf=True,
        norm='l2',
        ngram_range=(1, 2),  # Use unigrams and bigrams
        sublinear_tf=True,   # Apply sublinear TF scaling
        analyzer='word'      # You could also experiment with 'char' or 'char_wb' for character-level features
    )
    tfidf = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])
    return similarity[0][0]

In [ ]:
import spacy
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from collections import Counter
import numpy as np

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def preprocess_text(text):
    doc = nlp(text.lower())
    lemmatized_words = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        lemmatized_words.append(token.lemma_)
    return lemmatized_words

def expand_with_synonyms(words):
    expanded_words = words.copy()
    for word in words:
        expanded_words.extend(get_synonyms(word))
    return expanded_words

def calculate_enhanced_similarity(text1, text2):
    # Preprocess and tokenize texts
    words1 = preprocess_text(text1)
    words2 = preprocess_text(text2)

    # Expand with synonyms
    words1_expanded = expand_with_synonyms(words1)
    words2_expanded = expand_with_synonyms(words2)

    # Count word frequencies
    freq1 = Counter(words1_expanded)
    freq2 = Counter(words2_expanded)

    # Create a set of all unique words
    unique_words = set(freq1.keys()).union(set(freq2.keys()))

    # Create frequency vectors
    vector1 = [freq1[word] for word in unique_words]
    vector2 = [freq2[word] for word in unique_words]

    # Convert lists to numpy arrays
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)

    # Calculate cosine similarity
    cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

    return cosine_similarity


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import textwrap

paragraph = ' '.join(db_records)          # Joins the db_records text into a single line string
wrapper = textwrap.TextWrapper(width=80)  # Creates wrapper of 80 characters per line
text_wrapped = wrapper.fill(text = paragraph)

print(text_wrapped)

Retrieval Augmented Generation (RAG) represents a sophisticated hybrid approach
in the field of artificial intelligence, particularly within the realm of
natural language processing (NLP). It innovatively combines the capabilities of
neural network-based language models with retrieval systems to enhance the
generation of text, making it more accurate, informative, and contextually
relevant. This methodology leverages the strengths of both generative and
retrieval architectures to tackle complex tasks that require not only linguistic
fluency but also factual correctness and depth of knowledge. At the core of
Retrieval Augmented Generation (RAG) is a generative model, typically a
transformer-based neural network, similar to those used in models like GPT
(Generative Pre-trained Transformer) or BERT (Bidirectional Encoder
Representations from Transformers). This component is responsible for producing
coherent and contextually appropriate language outputs based on a mixture of
input prompts

## **Query**

In [ ]:
query = "define a rag store"

In [ ]:
# Call the function and print the result
llm_response = call_llm_with_full_text(query)
print_formatted_response(llm_response)

Response -------------------
It appears that the given content is a set of letters, each on a new line, forming a vertical arrangement. Let's break it down and analyze the possible meaning or interpretation:

1. **Vertical Arrangement**: The letters are arranged vertically, which could be a deliberate attempt to convey a message or create a visual effect. This arrangement might be used to draw attention to specific letters or to create a sense of rhythm.

2. **Individual Letters**: The letters are:
   - d
   - e
   - f
   - i
   - n
   - e
   - a
   - r
   - a
   - g
   - s
   - t
   - o
   - r
   - e

3. **Possible Words**: Upon closer inspection, it seems that these letters can be rearranged to form several words, including:
   - define
   - store
   - rage
   - age
   - torn
   - tore
   - east
   - dare
   - fine
   - fare

4. **Meaning and Context**: Without additional context, it's challenging to determine the intended meaning or message. However, some possible interpretations co

# **Advanced RAG**

## **Vector search**

### **Search Function**

In [ ]:
def find_best_match(text, records):
  best_score = 0
  best_record = None

  for record in records:
    current_score = calculate_cosine_similarity(text, record)

    if current_score > best_score:
      best_score = current_score
      best_record = record

  return best_score, best_record

In [ ]:
best_similarity_score, best_matching_record = find_best_match(query, db_records)
print_formatted_response(best_matching_record)

Response -------------------
A RAG vector store is a database or dataset that contains vectorized data points.
Formatted response -------------------
A RAG vector store is a database or dataset that contains vectorized data
points.


## **Metrics**

In [ ]:
print(f"Best Cosine Similarity Score: {best_similarity_score:.3f}")

Best Cosine Similarity Score: 0.126


In [ ]:
# Enhanced Similarity

response = best_matching_record
print(query,": ", response)
similarity_score = calculate_enhanced_similarity(query, best_matching_record)
print(f"Enhanced Similarity:, {similarity_score:.3f}")

define a rag store :  A RAG vector store is a database or dataset that contains vectorized data points.
Enhanced Similarity:, 0.642


## **Augmented Input**

In [ ]:
augmented_input = query+": "+best_matching_record

In [ ]:
print_formatted_response(augmented_input)

Response -------------------
define a rag store: A RAG vector store is a database or dataset that contains vectorized data points.
Formatted response -------------------
define a rag store: A RAG vector store is a database or dataset that contains
vectorized data points.


## **Generation**

In [ ]:
# Call the function and print the result
llm_response = call_llm_with_full_text(augmented_input)
print_formatted_response(llm_response)

Response -------------------
The given content appears to be a definition of a specific term, presented in a unique, spaced-out format. Let's break it down and read it in a more conventional way:

"Define a RAG vector store: A RAG vector store is a database or dataset that contains vectorized data points."

To elaborate on this, a RAG (Relational Algebra Graph) vector store is a type of data storage system that is designed to efficiently manage and query large amounts of vectorized data. Vectorized data refers to data that has been converted into a numerical representation, typically in the form of vectors or matrices, to enable efficient processing and analysis using machine learning algorithms.

In the context of Natural Language Processing (NLP), vectorized data can represent words, phrases, or documents as numerical vectors, allowing for semantic similarity calculations, clustering, and other types of analysis. A RAG vector store is optimized for storing and querying these vectoriz

While vector search significantly speeds up the process of finding relevant documents by sequentially going through each record, its efficiency can decrease as the dataset size increases. To address this scalability issue, indexed search offers a more advanced solution.

## **Index-based Search**

Index-based search compares the vector of a user query not with the direct vector of a document’s content but with an indexed vector that represents this content.

### **Search Function**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def setup_vectorizer(records):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(records)
    return vectorizer, tfidf_matrix

def find_best_match(query, vectorizer, tfidf_matrix):
    query_tfidf = vectorizer.transform([query])
    similarities = cosine_similarity(query_tfidf, tfidf_matrix)
    best_index = similarities.argmax()  # Get the index of the highest similarity score
    best_score = similarities[0, best_index]
    return best_score, best_index

vectorizer, tfidf_matrix = setup_vectorizer(db_records)

best_similarity_score, best_index = find_best_match(query, vectorizer, tfidf_matrix)
best_matching_record = db_records[best_index]

print_formatted_response(best_matching_record)

Response -------------------
A RAG vector store is a database or dataset that contains vectorized data points.
Formatted response -------------------
A RAG vector store is a database or dataset that contains vectorized data
points.


## **Metrics**

In [ ]:
# Cosine Similarity

print(f"Best Cosine Similarity Score: {best_similarity_score:.3f}")
print_formatted_response(best_matching_record)

Best Cosine Similarity Score: 0.407
Response -------------------
A RAG vector store is a database or dataset that contains vectorized data points.
Formatted response -------------------
A RAG vector store is a database or dataset that contains vectorized data
points.


In [ ]:
# Enhanced Similarity

response = best_matching_record
print(query,": ", response)
similarity_score = calculate_enhanced_similarity(query, response)
print(f"Enhanced Similarity:, {similarity_score:.3f}")

define a rag store :  A RAG vector store is a database or dataset that contains vectorized data points.
Enhanced Similarity:, 0.642


## **Feature extraction**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def setup_vectorizer(records):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(records)

    # Convert the TF-IDF matrix to a DataFrame for display purposes
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    # Display the DataFrame
    print(tfidf_df)

    return vectorizer, tfidf_matrix

vectorizer, tfidf_matrix = setup_vectorizer(db_records)

     ability    access  accuracy  accurate     adapt  additional  advancement  \
0   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
1   0.000000  0.000000  0.000000  0.216364  0.000000    0.000000     0.000000   
2   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
3   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.000000    0.236479     0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
7   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
10  0.186734  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
11  0.000000  0.000000  0.00

## **Augmented Input**

In [ ]:
augmented_input = query+": "+best_matching_record

In [ ]:
print_formatted_response(augmented_input)

Response -------------------
define a rag store: A RAG vector store is a database or dataset that contains vectorized data points.
Formatted response -------------------
define a rag store: A RAG vector store is a database or dataset that contains
vectorized data points.


## **Generation**

In [ ]:
# Call the function and print the result
llm_response = call_llm_with_full_text(augmented_input)
print_formatted_response(llm_response)

Response -------------------
The given content appears to be a definition of a specific data structure in the field of computer science and mathematics, particularly in the context of Natural Language Processing (NLP) and machine learning. Let's break it down:

**Definition:**

A RAG (Retrieval-Augmented Generator) vector store is a database or dataset that contains vectorized data points.

**Key Components:**

1. **RAG**: Stands for Retrieval-Augmented Generator, which is a type of neural network architecture that combines retrieval and generation capabilities.
2. **Vector Store**: A vector store is a database or dataset that stores vectorized data points, which are numerical representations of text, images, or other types of data.
3. **Vectorized Data Points**: These are numerical representations of text, images, or other types of data, which are used to train machine learning models or perform other tasks.

**Purpose:**

The purpose of a RAG vector store is to provide a repository o